In [4]:
import pandas as pd
import numpy as np
import MySQLdb
import datetime

In [2]:
company_order = pd.read_csv('myTempCompanyAll20170915_004.csv',encoding='utf-8-sig',header=None,index_col=None)

In [3]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [63]:
V_info_create()
B_info_create()
S_info_create()

In [17]:
now_date.strftime('%Y-%m-%d')

'2009-08-14'

In [14]:
now_date = datetime.datetime(2002,3,15)
last_date = datetime.datetime(2002,3,15)
end_date = datetime.datetime(2017,9,18)

databaseName = 'a_1_oi_original_data'

while now_date <= end_date:
    split_date = now_date.strftime('%Y-%m-%d').split('-')
    table_date = split_date[0]+split_date[1]+split_date[2]
    try:
        data = pd.read_csv('../大商所/大豆1号/'+table_date+'.csv',encoding='gbk',engine='python')
    except Exception as e:
        now_date = last_date + datetime.timedelta(days = 1)
        last_date = now_date
        print (e)
        continue
        
    for i in range(3,len(data)-1):
        for j in range(12):
            if(data.loc[i][j] == "-"):
                data.loc[i][j] = "NULL"                
            elif( (j==2 or j==3 or j==6 or j==7 or j==10 or j==11) and data.loc[i][j] != "NULL"):
                data.loc[i][j] = data.loc[i][j].replace(',','')
  
    tmp = now_date

    total_insert(data,table_date,databaseName)
    #futures_member_insert(data,table_date,databaseName)
    #non_futures_member_insert(data,table_date,databaseName)
    #total_volume_insert(data,table_date,databaseName)
        
    #V_info_insert(data,table_date,date['Date'][x].strftime('%Y-%m-%d'))
    #B_info_insert(data,table_date,date['Date'][x].strftime('%Y-%m-%d'))
    #S_info_insert(data,table_date,date['Date'][x].strftime('%Y-%m-%d'))
    
    now_date = last_date + datetime.timedelta(days = 1)
    last_date = now_date
    
db.close()

C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: Warning: (1050, "Table 'a_1_20020315' already exists")
  


[Errno 2] No such file or directory: '../大商所/大豆1号/20020316.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020317.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020323.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020324.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020330.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020331.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020406.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020407.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020413.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020414.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020420.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020421.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020427.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020428.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20020501.csv'
[Errno 2] No such file or directory: '..

[Errno 2] No such file or directory: '../大商所/大豆1号/20030405.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030406.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030412.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030413.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030419.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030420.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030426.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030427.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030501.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030502.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030503.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030504.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030505.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030506.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20030507.csv'
[Errno 2] No such file or directory: '..

[Errno 2] No such file or directory: '../大商所/大豆1号/20040417.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040418.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040424.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040425.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040501.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040502.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040503.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040504.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040505.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040506.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040507.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040508.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040509.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040515.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20040516.csv'
[Errno 2] No such file or directory: '..

[Errno 2] No such file or directory: '../大商所/大豆1号/20050514.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050515.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050521.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050522.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050528.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050529.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050604.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050605.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050611.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050612.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050618.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050619.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050625.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050626.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20050702.csv'
[Errno 2] No such file or directory: '..

[Errno 2] No such file or directory: '../大商所/大豆1号/20060610.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060611.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060617.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060618.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060624.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060625.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060701.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060702.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060708.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060709.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060715.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060716.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060722.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060723.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20060729.csv'
[Errno 2] No such file or directory: '..

[Errno 2] No such file or directory: '../大商所/大豆1号/20070707.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070708.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070714.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070715.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070721.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070722.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070728.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070729.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070804.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070805.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070811.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070812.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070818.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070819.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20070825.csv'
[Errno 2] No such file or directory: '..

[Errno 2] No such file or directory: '../大商所/大豆1号/20080809.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080810.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080816.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080817.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080823.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080824.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080830.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080831.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080906.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080907.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080913.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080914.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080915.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080920.csv'
[Errno 2] No such file or directory: '../大商所/大豆1号/20080921.csv'
[Errno 2] No such file or directory: '..

KeyboardInterrupt: 

In [13]:
def total_insert(data,table_date,databaseName):       
    cursor = db.cursor()
    
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".a_1_" + table_date+ "(\
    Rank INT(5),\
    Member CHAR(20),\
    Volume FLOAT,\
    Increment FLOAT,\
    Rank_1 INT(5),\
    Member1 CHAR(20),\
    Buy_Volume FLOAT,\
    Increment_1 FLOAT,\
    Rank_2 INT(5),\
    Member2 CHAR(20),\
    Sales_Volume FLOAT,\
    Increment_2 FLOAT )"

    cursor.execute(sql)

    for i in range(3,len(data)-1):
        
        No = data.loc[i][0]
        Member = data.loc[i][1]
        Volume = data.loc[i][2]
        Increase_or_decrease = data.loc[i][3]
        No1 = data.loc[i][4]
        Member1 = data.loc[i][5]
        Buy_Volume = data.loc[i][6]
        Increase_or_decrease1 = data.loc[i][7]
        No2 = data.loc[i][8]
        Member2 = data.loc[i][9]
        Sales_Volume = data.loc[i][10]
        Increase_or_decrease2 = data.loc[i][11]

        sql = "INSERT INTO " + databaseName + ".a_1_" + table_date +"(Rank,Member,Volume,Increment,Rank_1,Member1,Buy_Volume,Increment_1 \
        ,Rank_2,Member2,Sales_Volume,Increment_2) VALUES(" \
        + No + ",'" + Member + "'," + Volume + "," + Increase_or_decrease + ","\
        + No1 + ",'" + Member1 + "'," + Buy_Volume + "," + Increase_or_decrease1 + ","\
        + No2 + ",'" + Member2 + "'," + Sales_Volume + "," + Increase_or_decrease2 + ")"
        cursor.execute(sql)
        db.commit()
    cursor.close()

# 期貨公司成員輸入

In [5]:
def futures_member_insert(data,table_date,databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".futures_member (\
    Date DATE PRIMARY KEY,\
    Total_volume INT,\
    Increment INT,\
    Total_buy_volume INT,\
    Increment_1 INT,\
    Total_sales_volume INT,\
    Increment_2 INT ) "

    cursor.execute(sql)
    
    sql = "INSERT INTO " + databaseName + ".futures_member (Date,Total_volume,Increment,Total_buy_volume,Increment_1,Total_sales_volume,Increment_2) VALUES( "\
    + table_date +","\
    + data.loc[0][1].replace(',','') +","\
    + data.loc[0][2].replace(',','') +","\
    + data.loc[0][3].replace(',','') +","\
    + data.loc[0][4].replace(',','') +","\
    + data.loc[0][5].replace(',','') +","\
    + data.loc[0][6].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()
    cursor.close()

# 非期貨公司成員輸入

In [243]:
cursor = db.cursor()
sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".none_futures_member (\
Date DATE PRIMARY KEY,\
Total_volume INT,\
Increment INT,\
Total_buy_volume INT,\
Increment_1 INT,\
Total_sales_volume INT,\
Increment_2 INT ) "

cursor.execute(sql)

0

In [21]:
def non_futures_member_insert(data,table_date,databaseName):
    cursor = db.cursor()
    sql = "INSERT INTO " + databaseName +".none_futures_member (Date,Total_volume,Increment,Total_buy_volume,Increment_1,Total_sales_volume,Increment_2) VALUES("\
    + table_date + ","\
    + data.loc[1][1].replace(',','') +","\
    + data.loc[1][2].replace(',','') +","\
    + data.loc[1][3].replace(',','') +","\
    + data.loc[1][4].replace(',','') +","\
    + data.loc[1][5].replace(',','') +","\
    + data.loc[1][6].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()
    cursor.close()

# 總計

In [247]:
cursor = db.cursor()
sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".total_volume ( Date DATE PRIMARY KEY, \
V_Valume INT,V_Increment INT,\
B_Valume INT,B_Increment INT,\
S_Valume INT,S_Increment INT)"

cursor.execute(sql)

0

In [25]:
def total_volume_insert(data,table_date,databaseName):
    cursor = db.cursor()
    total_len = len(data)-1
    sql = "INSERT INTO " + databaseName + ".total_volume (Date,V_Valume,V_Increment,B_Valume,B_Increment,S_Valume,S_Increment) VALUES("\
    + table_date + ","\
    + data.loc[total_len][2].replace(',','') +","\
    + data.loc[total_len][3].replace(',','') +","\
    + data.loc[total_len][6].replace(',','') +","\
    + data.loc[total_len][7].replace(',','') +","\
    + data.loc[total_len][10].replace(',','') +","\
    + data.loc[total_len][11].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()

# 各公司資訊

## V名次、成交量、增減

In [45]:
def V_info_create():
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' CHAR(10),'
    name = name + company_order.loc[len(company_order)-1][0] + " CHAR(10)"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".v_rank ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_volume ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_increment ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)
    cursor.close()

In [92]:
def V_info_insert(data,table_date,list_date,databaseName):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ','
    name = name + company_order.loc[len(company_order)-1][0]

    empty = ""
    empty = empty + table_date + ","
    for i in range(len(company_order)-1):
        empty = empty + "NULL" + ','
    empty = empty + "NULL"

    sql = "INSERT INTO " + databaseName + ".v_rank (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_volume (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_increment (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
        if(data.loc[i][1] == "NULL"):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".v_rank SET " + str(data.loc[i][1]) + "=" + data.loc[i][0] + " WHERE Date='" + list_date + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][5] == "NULL"):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_volume SET " + str(data.loc[i][1]) + "=" + data.loc[i][2] + " WHERE Date='" + list_date + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][9] == "NULL"):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_increment SET " + data.loc[i][1] + "=" + data.loc[i][3] + " WHERE Date='" + list_date + "'"
            cursor.execute(sql)
            db.commit()
    
    cursor.close()

## B名次、買單量、增減

In [198]:
def B_info_create():
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' CHAR(10),'
    name = name + company_order.loc[len(company_order)-1][0] + " CHAR(10)"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_rank ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_oi ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_increment ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

0

In [199]:
def B_info_insert(data,table_date,list_date,databaseName)
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ','
    name = name + company_order.loc[len(company_order)-1][0]

    empty = ""
    empty = empty + table_date + ","
    for i in range(len(company_order)-1):
        empty = empty + "NULL" + ','
    empty = empty + "NULL"
    
    sql = "INSERT INTO " + databaseName + ".b_rank (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_oi (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_increment (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
    if(data.loc[i][1] == "NULL"):
        continue  
        
    sql = "UPDATE " + databaseName + ".b_rank SET " + str(data.loc[i][5]) + "=" + data.loc[i][4] + " WHERE Date='" + list_date + "'"
    cursor.execute(sql)
    db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][5] == "NULL"):
            continue
        sql = "UPDATE " + databaseName + ".b_oi SET " + str(data.loc[i][5]) + "=" + data.loc[i][6] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][9] == "NULL"):
            continue
        sql = "UPDATE " + databaseName + ".b_increment SET " + data.loc[i][5] + "=" + data.loc[i][7] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    cursor.close()

## S名次、賣單量、增減

In [203]:
def S_info_create():
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' CHAR(10),'
    name = name + company_order.loc[len(company_order)-1][0] + " CHAR(10)"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".s_rank ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_oi ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_increment ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

0

In [204]:
def S_info_insert(data,table_date,list_date,databaseName):
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ','
    name = name + company_order.loc[len(company_order)-1][0]

    empty = ""
    empty = empty + table_date + ","
    for i in range(len(company_order)-1):
        empty = empty + "NULL" + ','
    empty = empty + "NULL"
    
    sql = "INSERT INTO " + databaseName + ".s_rank (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_oi (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_increment (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
    if(data.loc[i][1] == "NULL"):
        continue  
        
    sql = "UPDATE " + databaseName + ".s_rank SET " + str(data.loc[i][1]) + "=" + data.loc[i][0] + " WHERE Date='" + list_date + "'"
    cursor.execute(sql)
    db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][5] == "NULL"):
            continue
        sql = "UPDATE " + databaseName + ".s_oi SET " + str(data.loc[i][1]) + "=" + data.loc[i][2] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][9] == "NULL"):
            continue
        sql = "UPDATE " + databaseName +".s_increment SET " + data.loc[i][9] + "=" + data.loc[i][3] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    cursor.close()